In [3]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)

import os
print("DB_HOST =", os.getenv("DB_HOST"))


DB_HOST = None


In [4]:
import os
print("cwd =", os.getcwd())
print("files =", os.listdir("."))


cwd = /Users/lancy/Desktop/:Users:lancy:Desktop:lancy_slash_project:
files = ['lancy_slash_sample.ipynb']


In [5]:
from dotenv import load_dotenv
load_dotenv("/Users/lancy/Desktop/.env", override=True)

import os
print(os.getenv("DB_HOST"))


db.ptukzshzuloxipzwycte.supabase.co


In [7]:
import os
import psycopg2
from dotenv import load_dotenv

# 加载 .env（用绝对路径，最稳）
load_dotenv("/Users/lancy/Desktop/.env", override=True)

# 创建连接
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT", "5432")),
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    sslmode="require",
)

print("✅ Database connected")


✅ Database connected


In [10]:
query = """
SELECT *
FROM fannie_mae_cas_raw
TABLESAMPLE SYSTEM (5)
LIMIT 300000;
"""
df = pd.read_sql(query, conn)
print(df.shape)



/var/folders/g1/xtt32sq56_b6b6bts7mwbmxr0000gn/T/ipykernel_14112/523815376.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(300000, 110)


In [ ]:
#1 
df.to_csv(
    "fannie_mae_sample_300k_raw.csv",
    index=False
)


In [13]:
df_clean = df.copy()

# 看缺失情况
missing = df_clean.isna().mean().sort_values(ascending=False)
print(missing.head(10))


interest_only_first_principal_and_interest_payment_date    1.0
months_to_amortization                                     1.0
arm_product_type                                           1.0
initial_fixed_rate_period                                  1.0
arm_initial_fixed_rate_period_5_yr_indicator               1.0
arm_cap_structure                                          1.0
lifetime_interest_rate_cap_up_percent                      1.0
periodic_interest_rate_cap_up_percent                      1.0
arm_plan_number                                            1.0
current_list_start_date                                    1.0
dtype: float64


In [14]:
# 删除 100% 缺失的列
cols_all_missing = missing[missing == 1.0].index
df_clean = df_clean.drop(columns=cols_all_missing)

print(f"Dropped {len(cols_all_missing)} columns with 100% missing values")


Dropped 20 columns with 100% missing values


In [15]:
missing[missing.between(0.05, 0.9)].head(20)


mortgage_insurance_percentage                    0.658193
mortgage_insurance_type                          0.658193
mortgage_insurance_cancellation_indicator        0.658193
co_borrower_credit_score_current                 0.595930
co_borrower_credit_score_at_issuance             0.576377
co_borrower_credit_score_at_origination          0.576003
interest_bearing_upb                             0.379347
payment_deferral_modification_event_indicator    0.337350
borrower_credit_score_current                    0.050990
dtype: float64

In [ ]:
from sklearn.impute import SimpleImputer

# ---------- 1️⃣ 明确列分组 ----------

# A. mortgage insurance 相关（缺失 = 无）
mi_cols = [
    "mortgage_insurance_percentage",
    "mortgage_insurance_type",
    "mortgage_insurance_cancellation_indicator",
]

# B. co-borrower credit score（数值）
co_borrower_score_cols = [
    "co_borrower_credit_score_current",
    "co_borrower_credit_score_at_issuance",
    "co_borrower_credit_score_at_origination",
]

# C. 其他数值型（用 median）
median_numeric_cols = [
    "interest_bearing_upb",
    "borrower_credit_score_current",
]

# ---------- 2️⃣ 分别填补 ----------

# A. mortgage insurance A 类：是否存在型变量（缺失本身有意义）0填
df_clean[mi_cols] = df_clean[mi_cols].fillna(
    {
        "mortgage_insurance_percentage": 0,
        "mortgage_insurance_type": "None",
        "mortgage_insurance_cancellation_indicator": 0,
    }
)

# B. co-borrower credit scores → median B 类：信用分数（数值型）median填
median_imputer = SimpleImputer(strategy="median")
df_clean[co_borrower_score_cols] = median_imputer.fit_transform(
    df_clean[co_borrower_score_cols]
)

# C. 其他数值型 → median C 类：金额型（UPB）median填
df_clean[median_numeric_cols] = median_imputer.fit_transform(
    df_clean[median_numeric_cols]
)

print("✅ Partial missing columns imputed")


✅ Partial missing columns imputed


In [17]:
df_clean[mi_cols + co_borrower_score_cols + median_numeric_cols].isna().mean()


mortgage_insurance_percentage                0.0
mortgage_insurance_type                      0.0
mortgage_insurance_cancellation_indicator    0.0
co_borrower_credit_score_current             0.0
co_borrower_credit_score_at_issuance         0.0
co_borrower_credit_score_at_origination      0.0
interest_bearing_upb                         0.0
borrower_credit_score_current                0.0
dtype: float64

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# 数值型 & 类别型列
num_cols = df_clean.select_dtypes(include=["int64", "float64"]).columns
cat_cols = df_clean.select_dtypes(include=["object", "category"]).columns

print("Numeric columns:", len(num_cols))
print("Categorical columns:", len(cat_cols))


Numeric columns: 5
Categorical columns: 85


In [19]:
from sklearn.preprocessing import StandardScaler
#数值列填median
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)


In [22]:
cat_cols = df_clean.select_dtypes(include=["object", "category"]).columns
print(f"Categorical columns ({len(cat_cols)}):")
cat_cols

for col in cat_cols:
    print(f"\n=== {col} ===")
    print(df_clean[col].value_counts(dropna=False).head(10))


Categorical columns (85):

=== reference_pool_id ===
reference_pool_id
5361    102658
5350    100795
5353     96319
5363       228
Name: count, dtype: int64

=== loan_identifier ===
loan_identifier
134318499    8
134248051    8
134248053    8
134303102    7
134248040    7
134248042    7
134248045    7
134248047    7
134248050    7
134248027    7
Name: count, dtype: int64

=== monthly_reporting_period ===
monthly_reporting_period
22025     12819
52023     12728
42024     12714
112024    12515
122024    12450
32024     12395
22023     12356
42025     12283
112023    12162
32023     12142
Name: count, dtype: int64

=== channel ===
channel
R    201427
C     58690
B     39883
Name: count, dtype: int64

=== seller_name ===
seller_name
Other                                        132974
Rocket Mortgage, LLC                          30400
United Wholesale Mortgage, LLC                16813
Wells Fargo Bank, N.A.                        11481
JPMorgan Chase Bank, National Association      8547
l

In [25]:
df_clean.select_dtypes(include=["object","category"]).nunique().sort_values(ascending=False).head(20)


current_actual_upb                               269759
loan_identifier                                  172551
upb_at_issuance                                  146289
total_principal_current                          107898
scheduled_principal_current                      101236
unscheduled_principal_current                     21204
upb_at_the_time_of_removal                         9708
loan_payment_history                               4248
original_interest_rate                             1578
current_interest_rate                              1555
original_upb                                       1002
zip_code_short                                      881
total_deferral_amount                               585
remaining_months_to_maturity                        455
metropolitan_statistical_area_msa                   393
borrower_credit_score_at_origination                231
remaining_months_to_legal_maturity                  196
maturity_date                                   

In [ ]:
#明确“真正的 categorical 能onehot的
cat_cols = (
    df_clean
    .select_dtypes(include=["object","category"])
    .columns
)

# 用 unique 数过滤
cat_cardinality = df_clean[cat_cols].nunique()

# 只保留低基数 categorical
LOW_CARD_MAX = 50
true_cat_cols = cat_cardinality[cat_cardinality <= LOW_CARD_MAX].index.tolist()

print("True categorical cols (for One-Hot):")
print(true_cat_cols)


True categorical cols (for One-Hot):
['reference_pool_id', 'monthly_reporting_period', 'channel', 'seller_name', 'servicer_name', 'master_servicer', 'origination_date', 'first_payment_date', 'original_loan_to_value_ratio_ltv', 'original_combined_loan_to_value_ratio_cltv', 'number_of_borrowers', 'first_time_home_buyer_indicator', 'loan_purpose', 'property_type', 'number_of_units', 'occupancy_status', 'mortgage_insurance_percentage', 'amortization_type', 'prepayment_penalty_indicator', 'interest_only_loan_indicator', 'current_loan_delinquency_status', 'modification_flag', 'mortgage_insurance_cancellation_indicator', 'zero_balance_code', 'zero_balance_effective_date', 'last_paid_installment_date', 'foreclosure_date', 'disposition_date', 'foreclosure_costs', 'property_preservation_and_repair_costs', 'asset_recovery_costs', 'miscellaneous_holding_expenses_and_credits', 'associated_taxes_for_holding_property', 'net_sales_proceeds', 'credit_enhancement_proceeds', 'repurchase_make_whole_procee

In [ ]:
#把“假 categorical”转回数值 or 删除
numeric_like_cols = [
    "current_actual_upb",
    "upb_at_issuance",
    "total_principal_current",
    "scheduled_principal_current",
    "unscheduled_principal_current",
    "original_interest_rate",
    "current_interest_rate",
    "original_upb",
]

for c in numeric_like_cols:
    df_clean[c] = pd.to_numeric(df_clean[c], errors="coerce")


In [ ]:
#id类全drop
id_like_cols = [
    "loan_identifier",
    "loan_payment_history",
]

df_clean = df_clean.drop(columns=id_like_cols)


In [59]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer

TARGET = "zero_balance_code"

# 1) 先构造 y 和 X（非常关键：列清单必须基于最终 X）
df_model = df_clean.copy()
z = pd.to_numeric(df_model[TARGET], errors="coerce")
y = ((~z.isna()) & (z != 0)).astype(int)

# 建议同时移除可能泄漏的日期列（可选但推荐）
leak_cols = ["zero_balance_effective_date", "zero_balance_code_change_date"]
drop_cols = [TARGET] + [c for c in leak_cols if c in df_model.columns]

X = df_model.drop(columns=drop_cols)

# 2) 在 X 上重新算列类型（不要用 df_clean）
num_cols = X.select_dtypes(include=["int64", "float64", "bool"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

print("num:", len(num_cols), "cat:", len(cat_cols))

# 3) 预处理（RF 不需要 scaler，所以数值只 impute 就够）
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("to_str", FunctionTransformer(lambda X: X.astype(str), validate=False)),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop"
)


num: 13 cat: 73


In [60]:
import pandas as pd
import numpy as np

TARGET = "zero_balance_code"

# 1) 先构造标签 y
df_model = df_clean.copy()
z = pd.to_numeric(df_model[TARGET], errors="coerce")

# 这里的规则你也可以改：非空且 != 0 视为 “发生 zero balance（还清/终止）”
y = ((~z.isna()) & (z != 0)).astype(int)
print("Label distribution:\n", y.value_counts())

# 2) 再构造特征 X（一定要把 target 列从特征里去掉）
X = df_model.drop(columns=[TARGET])

# 3) 关键：基于 X 重新计算列类型（不要用旧的 num_cols/cat_cols）
num_cols = X.select_dtypes(include=["int64", "float64", "bool"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

print("num cols:", len(num_cols), "cat cols:", len(cat_cols))
print("Does TARGET appear in cols lists?",
      TARGET in num_cols, TARGET in cat_cols)  # 应该都是 False


Label distribution:
 zero_balance_code
0    286014
1     13986
Name: count, dtype: int64
num cols: 13 cat cols: 75
Does TARGET appear in cols lists? False False


In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

TARGET = "zero_balance_code"

# 1) 构造 y
df_model = df_clean.copy()
z = pd.to_numeric(df_model[TARGET], errors="coerce")
y = ((~z.isna()) & (z != 0)).astype(int)

# 2) 构造 X（去掉 target + 建议去掉泄漏日期列）
leak_cols = ["zero_balance_effective_date", "zero_balance_code_change_date"]
drop_cols = [TARGET] + [c for c in leak_cols if c in df_model.columns]
X = df_model.drop(columns=drop_cols)

# 3) 列类型（必须基于 X）
num_cols = X.select_dtypes(include=["int64", "float64", "bool"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

# ✅ 关键硬修复：把所有 cat 列统一成字符串（彻底杜绝 int/str 混合）
X[cat_cols] = X[cat_cols].astype("string")
# 用一个明确的占位符处理缺失（这样 onehot 会把它当成一个类别）
X[cat_cols] = X[cat_cols].fillna("MISSING")

print("num:", len(num_cols), "cat:", len(cat_cols))

# 4) 预处理
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

cat_pipe = Pipeline([
    # 这里 imputer 可有可无（你已经 fillna 了），保留也没问题
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop"
)

# 5) 切分
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 6) Random Forest
rf = RandomForestClassifier(
    n_estimators=300,
    n_jobs=-1,
    random_state=42,
    class_weight="balanced",
    min_samples_leaf=50,
)

model = Pipeline([
    ("preprocess", preprocessor),
    ("rf", rf),
])

model.fit(X_train, y_train)
print("✅ Random Forest trained")


num: 13 cat: 73
✅ Random Forest trained


In [65]:
from sklearn.metrics import roc_auc_score, classification_report

proba = model.predict_proba(X_test)[:, 1]
print("ROC-AUC:", roc_auc_score(y_test, proba))
print(classification_report(y_test, model.predict(X_test)))



ROC-AUC: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     57203
           1       1.00      1.00      1.00      2797

    accuracy                           1.00     60000
   macro avg       1.00      1.00      1.00     60000
weighted avg       1.00      1.00      1.00     60000



In [ ]:
#321